In [1]:
# import modules
import tensorflow as tf
from tensorflow import keras as K
import pandas as pd
import numpy as np
import os

In [2]:
# set working directory. make sure metadata and images are in folders in the working directory
os.chdir('set_your_working_directory/')
os.getcwd()

'/home/scott/Dropbox/HEI_19_1_Deep_Learning_Models'

In [3]:
# check the GPUs, make sure they are not being used
!nvidia-smi

Thu Aug 24 16:08:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN Xp     Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   33C    P8    10W / 250W |    202MiB / 12194MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN Xp     Off  | 00000000:06:00.0 Off |                  N/A |
| 23%   

In [4]:
# Define the Root Mean Squared Error Loss Function
def rmse(y_true, y_pred):
    return K.backend.sqrt(K.backend.mean(K.backend.square(y_pred - y_true)))

In [ ]:
####################################
### loop does all 3 pollutants #####
####################################

In [5]:
# specify the city
city = input("Which city: to or mtl? ")

# specify the minimum number of monitoring days per road segment. 
min_number_of_monitoring_days = int(input("Set minimum number of monitoring days per road segment: "))
initial_learning_rate = float(input("Set initial learning rate: ")) # good to start with 0.0001
initial_learning_rate_str = "{0:.0e}".format(initial_learning_rate) + 'lr'

# set the max number of epochs to train. Good to start with at least 20. Then if it's working well, go to 100 and train overnight
num_epochs = input("Max number of epochs? ")
num_epochs = int(num_epochs)
if num_epochs > 50:
    print('!!! Are you sure you want to train up to ' + str(num_epochs) + 'epochs?')

# some details relevant to the metadata, this will help with pointing to folders, to columns in the metadata, and with naming files that will be saved (e.g. model training log)
image_type = "satellite"
zoom_1 = 'images_18'
zoom_2 = 'images_19'
image_file_name = 'sat_file'

# continuous outcome
tv_class_mode = 'raw'
test_class_mode = None 

# loop will train models for each pollutant. 'ufp', 'size' and 'bc' are the names of separate columns in the metadata that indicate the ufp number concentration, mean ufp size, and bc mass concentrations
pollutant = ['ufp', 'size', 'bc']


Which city: to or mtl? mtl
Set minimum number of monitoring days per road segment: 6
Set initial learning rate: 0.0001
Max number of epochs? 2


In [6]:
# loop will train models for each of the pollutants
for i in pollutant:
    
    # the metadata has a column for each pollutant measure (i.e., ufp_median, size_median, bc_median)
    if i == 'size':
        exposure = i + '_' + 'median'
    else:
        exposure = 'log_' + i + '_' + 'median' # ufp and bc concentrations have been log-transformed in the metadata. the ufp and bc concentration columns in the metadata are called "log_ufp_median" and "log_ufp_median" respectively
    
    # the metadata has a column that denotes the number of monitoring days per road segment for UFP/Size and BC
    if i == "bc":
        p_filter = 'days' + '_' + i
    if i == "ufp":
        p_filter = 'days' + '_' + i
    if i == "size":
        p_filter = 'days' + '_ufp'
    
    # read in the metadata. There were separate monitoring campaigns in Montreal and Toronto. This models each city separately
    # you may have a different fileplan, so you'll have to update the filepath and filename
    if city == "to":
        dat = pd.read_csv(filepath_or_buffer = "data_files/images/to_images/model_development_images/t100_new_image_metadata.csv")

    # you may have a different fileplan, so you'll have to update the filepath and filename
    if city == "mtl":
        dat = pd.read_csv(filepath_or_buffer = "data_files/images/mtl_images/model_development_images/m100_new_image_metadata.csv")

    # remove excess columns from metadata
    if i == 'size':
        dat = dat[[image_file_name, 'set_ghp6', p_filter, exposure, 'temp', 'hum', 'ws']]
    else:
        dat = dat[[image_file_name, 'set_ghp6', p_filter, exposure, i + '_' + 'median', 'temp', 'hum', 'ws']]

    # subset to observations where exposure is nonmissing
    dat = dat[dat[exposure].notnull()].reset_index(drop=True)

    dat[[exposure]] = dat[[exposure]].astype("float32")
    
    # there are satellite images at two zoom levels for each road segment. create a row in the metadata for each image
    s1, s2 = dat.copy(), dat.copy()

    s1[image_file_name] = city+'_' +zoom_1+'/' + s1[image_file_name]
    s2[image_file_name] = city+'_' +zoom_2+'/' + s2[image_file_name]

    dat = pd.concat([s1, s2])
       
    # remove observations below the cut off (i.e. any road segments that were monitored on fewer days will be removed from the analysis, pollution measures from these "low number of monitoring days" road segments are too temporally unstable to be useful)
    dat = dat[dat[p_filter] >= min_number_of_monitoring_days]

    # prepare the generators, these point to where the images are, specify preprocessing, and specify batch size
    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=K.applications.xception.preprocess_input,
                                                         horizontal_flip=True, 
                                                         vertical_flip = True)


    train_generator = generator.flow_from_dataframe(dataframe=dat.loc[dat['set_ghp6']=='train', [exposure, image_file_name]].reset_index(drop=True),
                                                    directory='data_files/images/'+city+'_images/model_development_images/'+image_type+'_view_100m/',      # you may have a different fileplan, so you'll have to update the filepath to the images you are using the train the models
                                                    x_col= image_file_name,
                                                    y_col=exposure,
                                                    #has_ext=True, #depreciated, not needed because we already have the extension on our filenames
                                                    class_mode=tv_class_mode,
                                                    target_size=(256, 256),
                                                    color_mode='rgb',
                                                    batch_size=32*4,
                                                    shuffle=True)

    validate_generator = generator.flow_from_dataframe(dataframe=dat.loc[dat['set_ghp6']=='validate', [exposure, image_file_name]].reset_index(drop=True),
                                                         directory='data_files/images/'+city+'_images/model_development_images/'+image_type+'_view_100m/',   # you may have a different fileplan, so you'll have to update the filepath to the images you are using the train the models
                                                         x_col= image_file_name,
                                                         y_col=exposure,
                                                         #has_ext=True, #depreciated, not needed because we already have the extension on our filenames
                                                         class_mode=tv_class_mode,
                                                         target_size=(256, 256),
                                                         color_mode='rgb',
                                                         batch_size=32*4,
                                                         shuffle=False)

    test_generator = generator.flow_from_dataframe(dataframe=dat.loc[dat['set_ghp6']=='test', [exposure, image_file_name]].reset_index(drop=True),
                                                   directory='data_files/images/'+city+'_images/model_development_images/'+image_type+'_view_100m/',   # you may have a different fileplan, so you'll have to update the filepath to the images you are using the train the models
                                                   x_col= image_file_name,
                                                   #has_ext=True, #depreciated, not needed because we already have the extension on our filenames
                                                   class_mode= test_class_mode,
                                                   target_size=(256, 256),
                                                   color_mode='rgb',
                                                   batch_size=32*4,
                                                   shuffle=False)

    # set callbacks, you may have a different fileplan, so you'll have to update the filepath and filename
    early_stopping = K.callbacks.EarlyStopping(monitor='val_rmse', patience=15)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.1, patience=5, mode='min', verbose=1)
    csv_logger = K.callbacks.CSVLogger('model_development/model_logs/'+city+'_'+image_type+'/'+city+', '+exposure+', '+p_filter+' min '+str(min_number_of_monitoring_days)+' days, '+image_type+', ' + initial_learning_rate_str + '.csv')
    model_checkpoint = K.callbacks.ModelCheckpoint('model_development/models/'+city+'_'+image_type+'/'+city+', '+exposure+', '+p_filter+' min '+str(min_number_of_monitoring_days)+' days, '+image_type+', ' + initial_learning_rate_str + '.hdf5', monitor='val_rmse', mode='min', save_weights_only=False, save_best_only=True)

    # define continous CNN model
    def get_compiled_model():
        # define model
        model_input = K.layers.Input(shape=(256, 256, 3), dtype='float32', name='input')
        conv_base = K.applications.Xception(include_top=False, weights='imagenet', input_tensor=model_input)
        model_output = K.layers.GlobalAveragePooling2D()(conv_base.output)
        model_output = K.layers.Dense(units=1, activation='linear')(model_output)
        model = K.models.Model(inputs=model_input, outputs=model_output)
        model.compile(
            optimizer=K.optimizers.Nadam(lr= initial_learning_rate),
            loss = rmse,   # or MeanSquaredError(), or K.losses.MeanAbsoluteError(reduction="auto", name="mean_absolute_error")
            metrics = [rmse,'mae']
        )
        return model

    # Create a MirroredStrategy
    strategy = tf.distribute.MirroredStrategy()
    print("Number of devices: {}".format(strategy.num_replicas_in_sync))

    # Open a strategy scope and compile the model
    with strategy.scope():
        model = get_compiled_model()

    # Train the model on all available devices
    for layer in model.layers: layer.trainable = True

    # train the model
    model.fit(train_generator, 
              validation_data=validate_generator,
              epochs=num_epochs, 
              steps_per_epoch=int(np.ceil(train_generator.samples/train_generator.batch_size)),
              validation_steps=int(np.ceil(validate_generator.samples/validate_generator.batch_size)),
              callbacks=[early_stopping, reduce_lr_on_plateau, csv_logger, model_checkpoint])

    # load model generate predictions in the test set, you may have a different fileplan, so you'll have to update the filepath and filename
    model = K.models.load_model('model_development/models/'+city+'_'+image_type+'/'+city+', '+exposure+', '+p_filter+' min '+str(min_number_of_monitoring_days)+' days, '+image_type+', ' + initial_learning_rate_str + '.hdf5', custom_objects={'rmse': rmse})

    results = dat.loc[dat['set_ghp6']=='test', [image_file_name, 'set_ghp6', 'temp', 'hum', 'ws', i + '_' + 'median', exposure]].copy().rename(columns={'file': 'File', 'set_ghp6': 'Set', 'temp': 'Temp', 'hum': 'Hum', 'ws': 'Wind_Speed', i + '_' + 'median': i + '_' + 'median', exposure: exposure}).reset_index(drop=True)

    results[exposure+'_pred'] = model.predict(x=test_generator, steps=int(np.ceil(test_generator.samples/test_generator.batch_size)))
    
    # save and show the results, you may have a different fileplan, so you'll have to update the filepath and filename
    results.to_csv(path_or_buf='model_development/model_predictions/'+city+'_'+image_type+'/'+city+', '+exposure+', '+p_filter+' '+str(min_number_of_monitoring_days)+'o, '+image_type+', ' + initial_learning_rate_str + '.csv', index=False)
    results


Found 9316 validated image filenames.
Found 2500 validated image filenames.
Found 2304 validated image filenames.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/de

In [ ]:
#############################################
######## Generate predictions ###############
#############################################

In [10]:
# specify the city, exposure and the images for training. Note 'size' is ufp mean size
city = input("Which city: to or mtl? ")
min_number_of_monitoring_days = int(input("Set minimum number of monitoring days per road segment: "))
initial_learning_rate = float(input("Model was trained using which initial learning rate? ")) # good to start with 0.0001
initial_learning_rate_str = "{0:.0e}".format(initial_learning_rate) + 'lr'

# this will help point to folders and metadata columns 
image_type = 'satellite'
zoom_angle_1 = 'images_18'
zoom_angle_2 = 'images_19'
image_file_name = 'sat_file'

# continuous outcome
tv_class_mode = 'raw'
test_class_mode = None 

# can generate predictions on all the model development images (i.e. for the monitoring sites) or for the prediction surface (i.e. the fishnet, all the 100 m x 100 m cells in the study area)
dev_or_pred_images = input("Use which set of images? prediction or development? ")   

# there are model development images (i.e. image paired with monitoring data) and model prediction images (i.e. the prediction surface aka the fishnet)
    # the two different groups of images were maintained in two different databases. The images themseleves have the same specifications (e.g., zoom level and resolution), but they were used for different purposes
    # depending on the choice made above, the code will generate predictions using the model development images or the prediction surface images
if dev_or_pred_images == 'prediction':
    pred_image_gis_file = 'fishnet'
    end_image_folder_name = 'fishnet' 
else:
    if dev_or_pred_images == 'development':
        pred_image_gis_file = '100m_new_dev' 
        end_image_folder_name = '100m'      

# generate predictions using the different models for each of the pollutants. 
pollutant = ['ufp', 'size', 'bc']

Which city: to or mtl? mtl
Set minimum number of monitoring days per road segment: 6
Model was trained using which initial learning rate? 0.0001
Use which set of images? prediction or development? development


In [11]:
for i in pollutant:

    # exposure and p_filter are in the model names
    if i == 'size':
        exposure = i + '_' + 'median'
    else:
        exposure = 'log_' + i + '_' + 'median'
    
    if i == "bc":
        p_filter = 'days' + '_' + i
    if i == "ufp":
        p_filter = 'days' + '_' + i
    if i == "size":
        p_filter = 'days' + '_ufp'
    
    # load in the metadata. there is metadata for each city, and there is the model development metadata (useful for model evaluation) and prediction surface metadata (which does not have observed pollution levels, but does have the lat and long of the image which is used to place the prediction in space)
    # you may have a different fileplan, so you'll have to update the filepath and filename
    if city == "to":
        if dev_or_pred_images == 'prediction':
            dat = pd.read_csv(filepath_or_buffer = "data_files/images/to_images/model_prediction_images/t_fishnet_image_metadata.csv")
        else:
            dat = pd.read_csv(filepath_or_buffer = "data_files/images/to_images/model_development_images/t100_new_image_metadata.csv")                
            dat.rename(columns = {'point_lon':'lon', 'point_lat':'lat'}, inplace = True)

    # you may have a different fileplan, so you'll have to update the filepath and filename
    if city == "mtl":
        if dev_or_pred_images == 'prediction':
            dat = pd.read_csv(filepath_or_buffer = "data_files/images/mtl_images/model_prediction_images/m_fishnet_image_metadata.csv")
        else:
            dat = pd.read_csv(filepath_or_buffer = "data_files/images/mtl_images/model_development_images/m100_new_image_metadata.csv")   
            dat.rename(columns = {'point_lon':'lon', 'point_lat':'lat'}, inplace = True)                

    # remove excess columns
    dat = dat[[image_file_name, 'site_id', 'lon', 'lat']]
    
    # will generate predictions on images of both zoom levels
    s1, s2 = dat.copy(), dat.copy()

    s1[image_file_name] = 'model_'+dev_or_pred_images+'_images/'+image_type+'_view_'+end_image_folder_name+'/'+city+'_' +zoom_angle_1+'/' + s1[image_file_name]
    s2[image_file_name] = 'model_'+dev_or_pred_images+'_images/'+image_type+'_view_'+end_image_folder_name+'/'+city+'_' +zoom_angle_2+'/' + s2[image_file_name]

    dat = pd.concat([s1, s2])

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=K.applications.xception.preprocess_input,
                                                         horizontal_flip=False, 
                                                         vertical_flip = False) 

    test_generator = generator.flow_from_dataframe(dataframe=dat,
                                                   directory='data_files/images/'+city+'_images/',    # you may have a different fileplan, so you'll have to update the filepath to the images you are using the train the models
                                                   x_col= image_file_name,
                                                   class_mode= test_class_mode,
                                                   target_size=(256, 256),
                                                   color_mode='rgb',
                                                   batch_size=32*4,
                                                   shuffle=False)

    # load model, you may have a different fileplan, so you'll have to update the filepath and filename
    model = K.models.load_model('model_development/models/'+city+'_'+image_type+'/'+city+', '+exposure+', '+p_filter+' min '+str(min_number_of_monitoring_days)+' days, '+image_type+', ' + initial_learning_rate_str + '.hdf5', custom_objects={'rmse': rmse})

    # create a results dataframe based on the metadata
    results = dat

    # generate predictions and include in results dataframe
    results[exposure+'_prediction'] = model.predict(x=test_generator, steps=int(np.ceil(test_generator.samples/test_generator.batch_size)))

    # save results as csv, you may have a different fileplan, so you'll have to update the filepath and filename
    results.to_csv(path_or_buf='model_development/model_predictions/'+city+'_'+pred_image_gis_file+'_'+image_type+'/'+city+', '+exposure+', '+p_filter+' min '+str(min_number_of_monitoring_days)+' days, '+image_type+', ' + initial_learning_rate_str + '.csv', index=False)



Found 30930 validated image filenames.
Found 30930 validated image filenames.
Found 30930 validated image filenames.
